In [1]:
'''
Tegar Imansyah
Lenet with tensorflow and keras

ref:
1. https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py
2. http://www.pyimagesearch.com/2016/08/01/lenet-convolutional-neural-network-in-python/
'''

from keras import applications
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import keras.preprocessing.image as image
import numpy as np
import matplotlib.pyplot as plt


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 12810+8303
nb_validation_samples = 6319+4639
epochs = 50
batch_size = 16
input_shape = (img_width, img_height, 3) # 3 Channel, di belakang karena menggunakan tensorflow's channel_last


Using TensorFlow backend.


In [5]:
# prepare data augmentation configuration
train_datagen = image.ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = image.ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

Found 21113 images belonging to 2 classes.
Found 10958 images belonging to 2 classes.


In [3]:
print("[INFO] drawing networks...")
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

print("[INFO] compiling...")
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

[INFO] drawing networks...
[INFO] compiling...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 146, 146, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 73, 73, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 341056)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               43655296  
_________________________________________________________________
dropout_4 (Dropout)          

In [ ]:
print("[INFO] training...")
# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

[INFO] training...
Epoch 1/50
1319/1319 [==============================] - 379s - loss: 0.6094 - acc: 0.6523 - val_loss: 0.5685 - val_acc: 0.7039
Epoch 2/50
1319/1319 [==============================] - 124s - loss: 0.5427 - acc: 0.7182 - val_loss: 0.5093 - val_acc: 0.7410
Epoch 3/50
1319/1319 [==============================] - 124s - loss: 0.4865 - acc: 0.7593 - val_loss: 0.4680 - val_acc: 0.7783
Epoch 4/50
1319/1319 [==============================] - 124s - loss: 0.4552 - acc: 0.7849 - val_loss: 0.4788 - val_acc: 0.7601
Epoch 5/50
1319/1319 [==============================] - 124s - loss: 0.4352 - acc: 0.8016 - val_loss: 0.4523 - val_acc: 0.7877
Epoch 6/50
1319/1319 [==============================] - 123s - loss: 0.4193 - acc: 0.8128 - val_loss: 0.4211 - val_acc: 0.8183
Epoch 7/50
1319/1319 [==============================] - 124s - loss: 0.4071 - acc: 0.8200 - val_loss: 0.4210 - val_acc: 0.8117
Epoch 8/50
1319/1319 [==============================] - 124s - loss: 0.3945 - acc: 0.8278 - 

# Main Program Sampai Disini

In [13]:
import time
millis = int(round(time.time() * 1000))
weight_path = 'person_small_' + str(millis) + '.h5'
model.save_weights(weight_path)

In [14]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=1,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/1
1319/1319 [==============================] - 124s - loss: 0.1940 - acc: 0.9243 - val_loss: 0.2886 - val_acc: 0.8945


In [15]:
def check(probs,answer):
    if(probs[0,0] == 0):
        print('Predict: No Victim, Actual: ' + answer)
    else:
        print('Predict: Victim, Actual: ' + answer)
        
def showtest():  
    import os, random
    for i in range(9):
        if (np.random.randint(1,2+1) == 1):
            chosen_file = random.choice(os.listdir("data/validation/pos"))
            img_path = 'data/validation/pos/' 
            img_path+= str(chosen_file)
            answer = 'Victim'
        else:
            chosen_file = random.choice(os.listdir("data/validation/neg"))
            img_path = 'data/validation/neg/' 
            img_path+= str(chosen_file)
            answer = 'No Victim'

        
        img = image.load_img(img_path, target_size=(150,150))
        # plt.imshow(img)
        # plt.show()
        print(img_path)
        img = image.img_to_array(img)
        img = img.reshape((1,) + img.shape)
        start_time = time.time()
        probs = model.predict(img)
        check(probs, answer)
        print('Predicted in %s seconds' % (time.time()-start_time))


showtest()

data/validation/neg/victim856_0_5028.jpeg
Predict: No Victim, Actual: No Victim
Predicted in 0.0619530677795 seconds
data/validation/pos/victim508_0_7184.jpeg
Predict: No Victim, Actual: Victim
Predicted in 0.00343489646912 seconds
data/validation/neg/victim686_0_3172.jpeg
Predict: No Victim, Actual: No Victim
Predicted in 0.00347709655762 seconds
data/validation/neg/victim729_0_6296.jpeg
Predict: No Victim, Actual: No Victim
Predicted in 0.00336313247681 seconds
data/validation/neg/victim828_0_8654.jpeg
Predict: No Victim, Actual: No Victim
Predicted in 0.00329995155334 seconds
data/validation/neg/victim828_0_8750.jpeg
Predict: No Victim, Actual: No Victim
Predicted in 0.00345301628113 seconds
data/validation/neg/victim753_0_5525.jpeg
Predict: No Victim, Actual: No Victim
Predicted in 0.00346612930298 seconds
data/validation/pos/victim418_0_6258.jpeg
Predict: No Victim, Actual: Victim
Predicted in 0.00365614891052 seconds
data/validation/pos/victim501_0_2036.jpeg
Predict: No Victim, A